<a href="https://colab.research.google.com/github/Piyushbalwani/OCR/blob/master/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json

In [4]:
!mkdir .kaggle
!mkdir marathiocr

In [5]:
# from kaggle, please generate token
key = ""
name = ""
api_token = {"username":name,"key":key}


In [6]:
with open('/content/.kaggle/kaggle.json', 'w+') as file:
    json.dump(api_token, file)

In [7]:
!chmod 600 /content/.kaggle/kaggle.json

In [8]:
import os
os.environ['KAGGLE_USERNAME'] = name

In [9]:
os.environ['KAGGLE_KEY'] = key

In [10]:
os.chdir('/content/marathiocr/')
!kaggle datasets download -d shalakadeore/handwritten-marathi-devanagari-characters

 97% 153M/158M [00:02<00:00, 55.2MB/s]
100% 158M/158M [00:02<00:00, 63.0MB/s]


In [11]:
import random
import numpy as np
import pandas as pd
# Visualisation imports
import matplotlib.pyplot as plt
import seaborn as sns
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Keras Imports - CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
import zipfile
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [72]:
pwd

'/content/marathiocr/Data_set/C1'

# Data Generator

In [55]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)  

In [56]:
train_generator = image_generator.flow_from_directory(batch_size=128,
                                                 directory=path,
                                                 shuffle=False,
                                                 target_size = (224, 224),
                                                 subset="training",
                                                 class_mode='categorical')

validation_generator = image_generator.flow_from_directory(batch_size=128,
                                                 directory=path,
                                                 shuffle=False,
                                                 target_size = (224, 224), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 4640 images belonging to 58 classes.
Found 1159 images belonging to 58 classes.


# NN Model

In [60]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 3)))

# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=58, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
# fit on data for 30 epochs
model.fit_generator(train_generator, epochs=30, validation_data=validation_generator)

# Transfer Learning

In [ ]:
from keras.applications import VGG16

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

In [64]:
from keras.utils import to_categorical
# extract train and val features
vgg_features_train = pretrained_model.predict(train_generator)
vgg_features_val = pretrained_model.predict(validation_generator)

In [70]:
# OHE target column
train_target = to_categorical(train_generator.labels)
val_target = to_categorical(validation_generator.labels)

In [69]:
# val_target = to_categorical(val.labels)
vgg_features_train.shape, vgg_features_val.shape

((4640, 7, 7, 512), (1159, 7, 7, 512))

In [71]:
model2 = Sequential()
model2.add(Flatten(input_shape=(7,7,512)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(58, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               2508900   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 58)                5858      
Total params: 2,515,158
Trainable params: 2,514,958
Non-trainable params: 200
_________________________________________________________________
Epoch 1/50
37/37 [==============================] - 2s 49ms/step - loss: 4.0083 - accuracy: 0.0429 - val_los